### Importing the libraries required

In [2]:
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge

### Reading and checking the data

In [3]:
train=pd.read_csv('train.csv')
train.head()
meal_info=pd.read_csv('meal_info.csv')
f_center=pd.read_csv('fulfilment_center_info.csv')
train=pd.merge(train,meal_info,left_on='meal_id',right_on='meal_id')
train=pd.merge(train,f_center,left_on='center_id',right_on='center_id')
train=pd.get_dummies(train,columns=['center_type','region_code'])
train.drop(columns=['id','center_id','meal_id','city_code','category','cuisine','week'],inplace=True)
train.head()

,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,op_area,center_type_TYPE_A,center_type_TYPE_B,center_type_TYPE_C,region_code_23,region_code_34,region_code_35,region_code_56,region_code_71,region_code_77,region_code_85,region_code_93
0,136.83,152.29,0,0,177,2.0,0,0,1,0,0,0,1,0,0,0,0
1,135.83,152.29,0,0,323,2.0,0,0,1,0,0,0,1,0,0,0,0
2,132.92,133.92,0,0,96,2.0,0,0,1,0,0,0,1,0,0,0,0
3,135.86,134.86,0,0,163,2.0,0,0,1,0,0,0,1,0,0,0,0
4,146.50,147.50,0,0,215,2.0,0,0,1,0,0,0,1,0,0,0,0


### Saving the models for all the splitted datasets 

In [4]:
l=meal_info.meal_id.unique() #check 
l

array([1885, 1993, 2539, 1248, 2631, 1311, 1062, 1778, 1803, 1198, 2707,
       1847, 1438, 2494, 2760, 2490, 1109, 2290, 1525, 2704, 1878, 2640,
       2577, 1754, 1971, 2306, 2139, 2826, 2664, 2569, 1230, 1207, 2322,
       2492, 1216, 1727, 1902, 1247, 2304, 1543, 1770, 2126, 1558, 2581,
       1962, 1571, 2956, 2104, 2444, 2867, 1445], dtype=int64)

In [7]:
for i in l[:]:
    filename=f'{i:.0f}'
    df= pd.read_csv(filename+'.csv')
    pipe1= joblib.load(filename+'.sav')
    df.drop(columns=['meal_id', 'week','diff'], inplace=True)
    df['checkout_price'], df['base_price'], df['op_area']=df['checkout_price']/200, df['base_price']/200, df['op_area']/4
    X= df.drop(columns=['num_orders']).values
    X.reshape(-1,16)
    df['y1']= pipe1.predict(X)
    #saving m=ml model
    X= df.drop(columns=['num_orders']).values
    Y= df.num_orders.values
    X.reshape(-1,17)
    Y.reshape(-1)
    pipe2= Pipeline([('poly', PolynomialFeatures(degree=2, include_bias=True)),('lin', BayesianRidge(tol= 1e-20, n_iter=300, normalize=True))])
    pipe2.fit(X,Y)
    joblib.dump(pipe2, (filename+'2.sav'))